# Irene-run

## Run irene for Kr MC events

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
import argparse

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math

In [ ]:
from invisible_cities.cities.irene import Irene, IRENE
from invisible_cities.cities.base_cities import S12Params as S12P
import invisible_cities.core.configure as conf
from invisible_cities.core.system_of_units_c import SystemOfUnits

In [ ]:
units = SystemOfUnits()

#### Number of events to run, input and output files

In [ ]:
nevt=10
file_in = os.environ['IC_DATA']  + '/NEXT_v0_8_09/dst_NEXT_v0_08_09_Kr_ACTIVE_5_0_7bar_RWF_10000.root.h5'
file_out = os.environ['IC_DATA']  + '/NEXT_v0_8_09/dst_NEXT_v0_08_09_Kr_ACTIVE_5_0_7bar_PMP_{}.root.h5'.format(nevt)

### Running irene, v1: defining parameters from constructor

#### S1 and S2 pars

In [ ]:
s1par = S12P(tmin=99*units.mus, tmax=101*units.mus, lmin=4, lmax=20, stride=4, rebin=False)
s2par = S12P(tmin=101*units.mus, tmax=1199*units.mus, lmin=80, lmax=200000, stride=40, rebin=True)

#### Irene instance

In [ ]:
files_in = glob(file_in)
files_in.sort()
irene = Irene(run_number  = 0,
              files_in    = files_in,
              file_out    = file_out,
              compression = 'ZLIB4',
              nprint      = 1,
              n_baseline  = 38000,
              thr_trigger = 5 * units.adc,
              n_MAU       = 100,
              thr_MAU     = 3.0*units.adc,
              thr_csum_s1 = 0.2*units.adc,
              thr_csum_s2 = 1.0*units.adc,
              n_MAU_sipm  = 100,
              thr_sipm    = 5.0*units.pes,
              s1_params   = s1par,
              s2_params   = s2par,
              thr_sipm_s2 = 30*units.pes)

#### Run

In [ ]:
t0 = time()
irene.run(nmax=nevt)
t1 = time()        
dt = t1 - t0

print("run {} in {} s, time/event = {}".format(nevt, dt, dt/nevt))

### Running Irene v2: defining parameters from setting functions

In [ ]:
irene = Irene(run_number=0)

# input files
files_in = glob(file_in)
files_in.sort()
irene.set_input_files(files_in)

# output file
irene.set_output_file(file_out)
irene.set_compression('ZLIB4')

# print frequency
irene.set_print(nprint=1)

# parameters of BLR
irene.set_blr(n_baseline  = 38000,
              thr_trigger = 5 * units.adc)

# parameters of calibrated sums
irene.set_csum(n_MAU = 100,
               thr_MAU = 3 * units.adc,
               thr_csum_s1 = 0.2 * units.pes,
               thr_csum_s2 = 1.0 * units.pes)

# MAU and thresholds for SiPms
irene.set_sipm(n_MAU_sipm= 100,
               thr_sipm=5 * units.pes)

# parameters for PMAP searches
irene.set_pmap_params(s1_params   = s1par,
                      s2_params   = s2par,
                      thr_sipm_s2 = 30 * units.pes)


t0 = time()

# run
nevt = irene.run(nmax=nevt, print_empty=True)
t1 = time()
dt = t1 - t0
print("run {} in {} s, time/event = {}".format(nevt, dt, dt/nevt))

### Running Irene v3: from configuration file

In [ ]:
CONF_file = os.environ['IC_CONFIG'] + '/irene_KrMC.conf'

In [ ]:
CONF_file

In [ ]:
%less /Users/jjgomezcadenas/Projects/ICCONF/irene_KrMC.conf

In [ ]:
IRENE(['IRENE', '-c', CONF_file])